# Question 5

In [10]:
import random
import time
import numpy as np

# SimplifiedYannakakis algorithm from Problem 2
class SimplifiedYannakakis:
    def __init__(self, relations):
        self.relations = relations

    def perform_semi_join(self, r1, r2):
        join_attribute_r1 = {t[1] for t in r1}
        join_attribute_r2 = {t[0] for t in r2}
        semi_join_result = join_attribute_r1.intersection(join_attribute_r2)
        return [t for t in r1 if t[1] in semi_join_result]

    def reduce_relations(self):
        for i in reversed(range(len(self.relations) - 1)):
            self.relations[i] = self.perform_semi_join(self.relations[i], self.relations[i + 1])

    def execute_join(self):
        self.reduce_relations()
        join_result = self.relations[0]
        for i in range(1, len(self.relations)):
            next_relation = self.relations[i]
            join_result = [(t1[0],) + t1[1:] + t2[1:] for t1 in join_result for t2 in next_relation if t1[-1] == t2[0]]
        return join_result

# Line join algorithm from Problem 3
def hash_relation(relation, join_attribute_index):
    hash_map = {}
    for tuple_ in relation:
        key = tuple_[join_attribute_index]
        hash_map.setdefault(key, []).append(tuple_)
    return hash_map

def line_join(relations):
    join_result = relations[0]
    for i in range(1, len(relations)):
        hash_map = hash_relation(relations[i], 0)
        new_join_result = []
        for tuple_ in join_result:
            join_key = tuple_[-1]
            for join_tuple in hash_map.get(join_key, []):
                new_join_result.append(tuple_ + join_tuple[1:])
        join_result = new_join_result
    return join_result

def create_dataset_R1():
    tuples_1_1000 = [(i, 5) for i in range(1, 1001)]
    tuples_1001_2000 = [(i, 7) for i in range(1001, 2001)]
    tuples_extra = [(2001, 2002)]
    R1 = tuples_1_1000 + tuples_1001_2000 + tuples_extra
    np.random.shuffle(R1)
    return R1

def create_dataset_R2():
    tuples_1_1000 = [(5, i) for i in range(1, 1001)]
    tuples_1001_2000 = [(7, i) for i in range(1001, 2001)]
    tuples_extra = [(2002, 8)]
    R2 = tuples_1_1000 + tuples_1001_2000 + tuples_extra
    np.random.shuffle(R2)
    return R2

def create_dataset_R3():
    random_tuples = [(np.random.randint(2002, 3001), np.random.randint(1, 3001)) for _ in range(2000)]
    tuples_extra = [(8, 30)]
    R3 = random_tuples + tuples_extra
    np.random.shuffle(R3)
    return R3

R1 = create_dataset_R1()
R2 = create_dataset_R2()
R3 = create_dataset_R3()

yannakakis = SimplifiedYannakakis([R1, R2, R3])
start_time_yannakakis = time.time()
yannakakis_result = yannakakis.execute_join()
end_time_yannakakis = time.time()

start_time_line_join = time.time()
line_join_result = line_join([R1, R2, R3])
end_time_line_join = time.time()

execution_time_yannakakis = end_time_yannakakis - start_time_yannakakis
execution_time_line_join = end_time_line_join - start_time_line_join
results_same = yannakakis_result == line_join_result

print(f"Yannakakis Execution Time: {execution_time_yannakakis} seconds")
print(f"Line Join Execution Time: {execution_time_line_join} seconds")
print(f"Are results the same: {results_same}")
print(f"First 5 results from Yannakakis: {yannakakis_result[:10]}")
print(f"First 5 results from Line Join: {line_join_result[:10]}")


Yannakakis Execution Time: 0.21262717247009277 seconds
Line Join Execution Time: 0.6166152954101562 seconds
Are results the same: True
First 5 results from Yannakakis: [(567, 5, 8, 30), (383, 5, 8, 30), (624, 5, 8, 30), (737, 5, 8, 30), (274, 5, 8, 30), (868, 5, 8, 30), (772, 5, 8, 30), (601, 5, 8, 30), (594, 5, 8, 30), (88, 5, 8, 30)]
First 5 results from Line Join: [(567, 5, 8, 30), (383, 5, 8, 30), (624, 5, 8, 30), (737, 5, 8, 30), (274, 5, 8, 30), (868, 5, 8, 30), (772, 5, 8, 30), (601, 5, 8, 30), (594, 5, 8, 30), (88, 5, 8, 30)]


# Question 6

In [11]:
import mysql.connector
from mysql.connector import Error
import numpy as np

def insert_tuples(connection, table_name, tuples):
    cursor = connection.cursor()
    query = f"INSERT INTO {table_name} (column1, column2) VALUES (%s, %s)"
    try:
        cursor.executemany(query, tuples)
        connection.commit()
        print(f"Tuples inserted successfully into {table_name}")
    except Error as err:
        print(f"Error: '{err}'")

try:
    # Connect to the MySQL database
    connection = mysql.connector.connect(host='localhost',
                                         database='university',
                                         user='surya',
                                         password='Hello@123')
    if connection.is_connected():
        cursor = connection.cursor()

        create_table_R1 = """
        CREATE TABLE IF NOT EXISTS R1 (
            column1 INT,
            column2 INT
        );
        """
        create_table_R2 = """
        CREATE TABLE IF NOT EXISTS R2 (
            column1 INT,
            column2 INT
        );
        """
        create_table_R3 = """
        CREATE TABLE IF NOT EXISTS R3 (
            column1 INT,
            column2 INT
        );
        """

        cursor.execute(create_table_R1)
        cursor.execute(create_table_R2)
        cursor.execute(create_table_R3)
        connection.commit()
        print("Tables R1, R2, and R3 created successfully")

        insert_tuples(connection, 'R1', R1)
        insert_tuples(connection, 'R2', R2)
        insert_tuples(connection, 'R3', R3)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Tables R1, R2, and R3 created successfully
Tuples inserted successfully into R1
Tuples inserted successfully into R2
Tuples inserted successfully into R3
MySQL connection is closed
